### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Pheochromocytoma_and_Paraganglioma/GSE39198'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)
    
 
from numpy import nan


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = False  # Based on the dataset description, it is methylation data

# Age and gender keys from the Sample Characteristics Dictionary
sample_characteristics = {
    0: ['tissue: paraganglioma', 'tissue: normal adrenal medulla'],
    1: ['Sex: female', 'Sex: male'],
    2: ['malignancy: benign', 'mutation (gene): NF1', 'mutation (gene): VHL', 'mutation (gene): RET', 'mutation (gene): SDHD', 'mutation (gene): SDHB', 'mutation (gene): TMEM127', 'malignancy: malignant', 'mutation (gene): MAX', 'malignancy: NA', 'mutation (gene): SDHA', 'mutation (gene): SDHC', nan],
    3: [nan, 'mutation (type): germline', 'mutation (type): somatic', 'mutation (type): NA', 'mutation (type): syndromic'],
    4: [nan, 'malignancy: benign', 'malignancy: malignant']
}

# Define rows if variables are available
trait_row = 2  # malignancy status can be used as a representative trait
age_row = None  # age not listed
gender_row = 1

# Conversion functions for each variable
def convert_trait(value):
    try:
        return {'benign': 0, 'malignant': 1}[value.split(': ')[1]]
    except KeyError:
        return None

def convert_age(value):
    try:
        return int(value.split(': ')[1])
    except ValueError:
        return None

def convert_gender(value):
    try:
        return {'female': 0, 'male': 1}[value.split(': ')[1]]
    except KeyError:
        return None

# Save metadata information
save_cohort_info('GSE39198', './preprocessed/Pheochromocytoma_and_Paraganglioma/cohort_info.json', is_gene_available, trait_row is not None)

# Proceed with the clinical feature extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Pheochromocytoma_and_Paraganglioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Pheochromocytoma_and_Paraganglioma/trait_data/GSE39198.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
